In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/cropDisease/model2'

/content/drive/MyDrive/cropDisease/model2


In [4]:
import kagglehub
path = kagglehub.dataset_download("nirmalsankalana/crop-pest-and-disease-detection")
print("Path to dataset files:", path)

100%|██████████| 1.25G/1.25G [00:13<00:00, 96.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nirmalsankalana/crop-pest-and-disease-detection/versions/1


In [5]:
%ls '/root/.cache/kagglehub/datasets/nirmalsankalana/crop-pest-and-disease-detection/versions/1'

'Cashew anthracnose'/        'Maize grasshoper'/
'Cashew gumosis'/            'Maize healthy'/
'Cashew healthy'/            'Maize leaf beetle'/
'Cashew leaf miner'/         'Maize leaf blight'/
'Cashew red rust'/           'Maize leaf spot'/
'Cassava bacterial blight'/  'Maize streak virus'/
'Cassava brown spot'/        'Tomato healthy'/
'Cassava green mite'/        'Tomato leaf blight'/
'Cassava healthy'/           'Tomato leaf curl'/
'Cassava mosaic'/            'Tomato septoria leaf spot'/
'Maize fall armyworm'/       'Tomato verticulium wilt'/


In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to original data
original_dataset_path = path

# New YOLO classification dataset structure
output_path = '/content/crop_yolo'
train_path = os.path.join(output_path, 'train')
val_path = os.path.join(output_path, 'val')

# Create folders
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# Split and move data
for category in os.listdir(original_dataset_path):
    category_path = os.path.join(original_dataset_path, category)
    if not os.path.isdir(category_path):
        continue

    images = [f for f in os.listdir(category_path) if f.endswith('.jpg')]
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(val_path, category), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(category_path, img), os.path.join(train_path, category, img))
    for img in val_imgs:
        shutil.copy(os.path.join(category_path, img), os.path.join(val_path, category, img))

print("✅ Dataset successfully converted to YOLO classification format!")


✅ Dataset successfully converted to YOLO classification format!


In [7]:
!pip install ultralytics --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00


In [12]:
from ultralytics import YOLO

# Load YOLOv8 classification model
model = YOLO('yolov8n-cls.pt')  # You can use yolov8n-cls.pt / yolov8s-cls.pt etc.

# Train the model
model.train(data='/content/crop_yolo', epochs=15, imgsz=128, batch=32)

# Evaluate on validation set
metrics = model.val()

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/crop_yolo, epochs=15, time=None, patience=100, batch=32, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=N

train: Scanning /content/crop_yolo/train... 20103 images, 65 corrupt: 100%|██████████| 20103/20103 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/crop_yolo/train/Maize healthy/healthy189_.jpg: ignoring corrupt image/label: broken data stream when reading image file
train: WARNING ⚠️ /content/crop_yolo/train/Maize healthy/healthy18_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/train/Maize healthy/healthy18_.jpg'
train: WARNING ⚠️ /content/crop_yolo/train/Maize healthy/healthy87_.jpg: ignoring corrupt image/label: broken data stream when reading image file
train: WARNING ⚠️ /content/crop_yolo/train/Maize healthy/healthy88_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/train/Maize healthy/healthy88_.jpg'
train: WARNING ⚠️ /content/crop_yolo/train/Maize leaf beetle/leaf beetle325_.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/crop_yolo/train/Maize leaf beetle/leaf beetle326_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/train/Maize leaf beetle/leaf beetle326_.jpg'
train: WARNING ⚠️


val: Scanning /content/crop_yolo/val... 5043 images, 9 corrupt: 100%|██████████| 5043/5043 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle208_.jpg: ignoring corrupt image/label: Truncated File Read
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot271_.jpg: ignoring corrupt image/label: broken data stream when reading image file
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot798_.jpg: ignoring corrupt image/label: broken data stream when reading ima

...

                   all      0.875      0.999
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5
Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,463,062 parameters, 0 gradients, 3.3 GFLOPs
train: /content/crop_yolo/train... found 20168 images in 22 classes ✅ 
val: /content/crop_yolo/val... found 5052 images in 22 classes ✅ 
test: None...


val: Scanning /content/crop_yolo/val... 5043 images, 9 corrupt: 100%|██████████| 5043/5043 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle208_.jpg: ignoring corrupt image/label: Truncated File Read
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot271_.jpg: ignoring corrupt image/label: broken data stream when reading image file
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot798_.jpg: ignoring corrupt image/label: broken data stream when reading ima


               classes   top1_acc   top5_acc: 100%|██████████| 158/158 [00:14<00:00, 10.59it/s]


                   all      0.874      0.999
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train52


In [13]:
# Evaluate on validation set
metrics = model.val()

Ultralytics 8.3.108 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
train: /content/crop_yolo/train... found 20168 images in 22 classes ✅ 
val: /content/crop_yolo/val... found 5052 images in 22 classes ✅ 
test: None...


val: Scanning /content/crop_yolo/val... 5043 images, 9 corrupt: 100%|██████████| 5043/5043 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle207_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle208_.jpg: ignoring corrupt image/label: Truncated File Read
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf beetle/leaf beetle691_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot271_.jpg: ignoring corrupt image/label: broken data stream when reading image file
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg: ignoring corrupt image/label: cannot identify image file '/content/crop_yolo/val/Maize leaf spot/leaf spot724_.jpg'
val: WARNING ⚠️ /content/crop_yolo/val/Maize leaf spot/leaf spot798_.jpg: ignoring corrupt image/label: broken data stream when reading ima


               classes   top1_acc   top5_acc: 100%|██████████| 158/158 [00:14<00:00, 10.63it/s]


                   all      0.874      0.999
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train53
